# Introduction to Linear Regression with PyTorch

In [1]:
import numpy as np
import torch

## Training data

We can represent the training data using two matrices: inputs and targets, each with one row per observation, and one column per variable.



In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

We've separated the input and target variables because we'll operate on them separately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then convert them to PyTorch tensors.

Let's convert the arrays to PyTorch tensors.

In [4]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression model from scratch

The weights and biases (w11, w12,... w23, b1 & b2) can also be represented as matrices, initialized as random values. The first row of w and the first element of b are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

In [5]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 1.0678,  1.5871, -0.6977],
        [ 1.1371, -0.7533,  0.0850]], requires_grad=True)
tensor([1.4050, 1.3311], requires_grad=True)


Our model is simply a function that performs a matrix multiplication of the inputs and the weights w (transposed) and adds the bias b (replicated for each observation).


We can define the model as follows:



In [6]:
def model(x):
    return x @ w.t() + b

@ represents matrix multiplication in PyTorch, and the .t method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [7]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[155.6914,  37.5200],
        [193.5898,  43.9529],
        [266.5105,   4.2417],
        [152.7548,  88.0643],
        [178.6079,  13.4211]], grad_fn=<AddBackward0>)


Let's compare the predictions of our model with the actual targets.

In [8]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to just work.

---

# Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

Calculate the difference between the two matrices (preds and targets). \
Square all elements of the difference matrix to remove negative values. \
Calculate the average of the elements in the resulting matrix. \
The result is a single number, known as the mean squared error (MSE).

In [9]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

torch.sum returns the sum of all the elements in a tensor. The .numel method of a tensor returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.



In [11]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(10183.0049, grad_fn=<DivBackward0>)


Here’s how we can interpret the result: On average, each element in the prediction differs from the actual target by the square root of the loss. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. The result is called the loss because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

---

# Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have requires_grad set to True. We'll see how this is useful in just a moment.

In [13]:
#  Compute gradients
loss.backward()

The gradients are stored in the .grad property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix with the same dimensions.

In [14]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 1.0678,  1.5871, -0.6977],
        [ 1.1371, -0.7533,  0.0850]], requires_grad=True)
tensor([[ 9782.0986,  9846.8887,  6035.7139],
        [-4168.1377, -6477.9463, -3603.3564]])


# Adjust weights and biases to reduce the loss

The loss is a quadratic function of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's slope w.r.t. the weights and biases.

If a gradient element is positive: 

increasing the weight element's value slightly will increase the loss. \
decreasing the weight element's value slightly will decrease the loss.


If a gradient element is negative:

increasing the weight element's value slightly will decrease the loss
decreasing the weight element's value slightly will increase the loss

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of the gradient descent optimization algorithm that we'll use to improve our model (by descending along the gradient).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [15]:
w
w.grad

tensor([[ 9782.0986,  9846.8887,  6035.7139],
        [-4168.1377, -6477.9463, -3603.3564]])

In [16]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

We multiply the gradients with a very small number (10^-5 in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the learning rate of the algorithm.

We use torch.no_grad to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [17]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)

tensor(10183.0049, grad_fn=<DivBackward0>)


Before we proceed, we reset the gradients to zero by invoking the .zero_() method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke .backward on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [18]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


---

# Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can train the model using the following steps:

Generate predictions

Calculate the loss

Compute gradients w.r.t the weights and biases

Adjust the weights by subtracting a small quantity proportional to the gradient

Reset the gradients to zero

Let's implement the above step by step.

In [19]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[139.3565,  46.4530],
        [172.1589,  55.7532],
        [241.3033,  18.6389],
        [136.3086,  96.4351],
        [158.1791,  25.0389]], grad_fn=<AddBackward0>)


In [20]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(7436.8960, grad_fn=<DivBackward0>)


In [21]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ 8097.5410,  8041.5957,  4920.8096],
        [-3241.3501, -5469.6641, -2983.8887]])
tensor([ 93.2613, -43.5362])


Let's update the weights and biases using the gradients computed above.

In [22]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

Let's take a look at the new weights and biases.

In [23]:
print(w)
print(b)

tensor([[ 0.8890,  1.4082, -0.8073],
        [ 1.2112, -0.6338,  0.1509]], requires_grad=True)
tensor([1.4030, 1.3321], requires_grad=True)


With the new weights and biases, the model should have a lower loss.

In [24]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(5579.6694, grad_fn=<DivBackward0>)


We have already achieved a significant reduction in the loss merely by adjusting the weights and biases slightly using gradient descent.



# Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.

In [25]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

Once again, let's verify that the loss is now lower:

In [26]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(593.8553, grad_fn=<DivBackward0>)


The loss is now much lower than its initial value. Let's look at the model's predictions and compare them with the targets.

In [27]:
# Predictions
preds

tensor([[ 61.7920,  79.8277],
        [ 73.0180, 104.3097],
        [132.1576, 109.3824],
        [ 45.3742,  90.7653],
        [ 72.0339,  94.3652]], grad_fn=<AddBackward0>)

In [28]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

The predictions are now quite close to the target variables. We can get even better results by training for a few more epochs.

---

# Linear Regression using the PyTorch Built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the torch.nn package from PyTorch, which contains utility classes for building neural networks.

In [5]:
import torch
import torch.nn as nn

As before, we represent the inputs and targets and matrices.



In [6]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

We are using 15 training examples to illustrate how to work with large datasets in small batches.

# Dataset and DataLoader

We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [8]:
from torch.utils.data import TensorDataset

In [13]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The TensorDataset allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a DataLoader, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [14]:
from torch.utils.data import DataLoader

In [15]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

We can use the data loader in a for loop. Let's look at an example.

In [16]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[91., 87., 65.],
        [68., 97., 70.],
        [73., 67., 43.],
        [91., 88., 64.],
        [73., 66., 44.]])
tensor([[ 80., 102.],
        [102., 120.],
        [ 56.,  70.],
        [ 81., 101.],
        [ 57.,  69.]])


In each iteration, the data loader returns one batch of data with the given batch size. If shuffle is set to True, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

# nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [17]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.4289, -0.0651, -0.0455],
        [-0.0232,  0.1788, -0.5674]], requires_grad=True)
Parameter containing:
tensor([ 0.2728, -0.5043], requires_grad=True)


PyTorch models also have a helpful .parameters method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [18]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.4289, -0.0651, -0.0455],
         [-0.0232,  0.1788, -0.5674]], requires_grad=True),
 Parameter containing:
 tensor([ 0.2728, -0.5043], requires_grad=True)]

In [19]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 25.2622, -14.6133],
        [ 30.6589, -23.1905],
        [ 26.2209, -11.4687],
        [ 39.5366, -16.1720],
        [ 20.4288, -24.6548],
        [ 25.7562, -14.8153],
        [ 30.6785, -23.9367],
        [ 26.6042, -12.0592],
        [ 39.0425, -15.9700],
        [ 19.9544, -25.1990],
        [ 25.2818, -15.3595],
        [ 31.1530, -23.3925],
        [ 26.2013, -10.7225],
        [ 40.0110, -15.6278],
        [ 19.9348, -24.4528]], grad_fn=<AddmmBackward>)

# Loss Function

Instead of defining a loss function manually, we can use the built-in loss function mse_loss.

In [20]:
# Import nn.functional
import torch.nn.functional as F

The nn.functional package contains many useful loss functions and several other utilities.

In [21]:
# Define loss function
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [22]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(8617.3789, grad_fn=<MseLossBackward>)


# Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer optim.SGD. SGD is short for "stochastic gradient descent". The term stochastic indicates that samples are selected in random batches instead of as a single group.

In [23]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that model.parameters() is passed as an argument to optim.SGD so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

# Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1) Generate predictions

2) Calculate the loss

3) Compute gradients w.r.t the weights and biases

4) Adjust the weights by subtracting a small quantity proportional to the gradient

5) Reset the gradients to zero

The only change is that we'll work in batches of data instead of processing the entire training data in every iteration. Let's define a utility function fit that trains the model for a given number of epochs.

In [24]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

We use the data loader defined earlier to get batches of data for every iteration.

Instead of updating parameters (weights and biases) manually, we use opt.step to perform the update and opt.zero_grad to reset the gradients to zero.

We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. loss.item returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [25]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 860.4821
Epoch [20/100], Loss: 645.4150
Epoch [30/100], Loss: 133.7767
Epoch [40/100], Loss: 97.0501
Epoch [50/100], Loss: 163.8330
Epoch [60/100], Loss: 22.4807
Epoch [70/100], Loss: 102.5700
Epoch [80/100], Loss: 93.9731
Epoch [90/100], Loss: 101.6610
Epoch [100/100], Loss: 40.3530


Let's generate predictions using our model and verify that they're close to our targets.

In [26]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.5944,  71.4743],
        [ 80.2703,  94.9499],
        [118.6364, 141.8842],
        [ 30.5516,  45.3724],
        [ 93.2350, 104.3708],
        [ 57.5474,  70.4183],
        [ 79.7400,  94.0241],
        [118.7810, 141.9665],
        [ 31.5986,  46.4284],
        [ 93.7517, 104.5010],
        [ 58.0641,  70.5485],
        [ 79.2233,  93.8938],
        [119.1667, 142.8100],
        [ 30.0349,  45.2422],
        [ 94.2820, 105.4268]], grad_fn=<AddmmBackward>)

In [27]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [28]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[55.1090, 67.4044]], grad_fn=<AddmmBackward>)

The predicted yield of apples is 54.3 tons per hectare, and that of oranges is 68.3 tons per hectare.